<a href="https://colab.research.google.com/github/sunghwan-moon/2021_capstone_BrainAgePrediction_by_Federated_Learning/blob/main/FL_Simple_Data_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install tensorflow_federated
#!pip install nilearn
#!pip install transformations
#!pip install nest_asyncio

In [ ]:
import collections
from nilearn import image
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import threading
from transformations import rotation_matrix
from random import gauss
from scipy.ndimage.interpolation import map_coordinates
import operator
from scipy.ndimage.interpolation import shift,rotate
import pandas as pd
import os
import scipy.ndimage as nd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_absolute_error
import numpy as np
import matplotlib.pyplot as plt
import math
import sys
import random
import glob
import nibabel as nib
from collections import defaultdict
from tensorflow.keras.utils import Sequence
import nest_asyncio
nest_asyncio.apply()
#from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input, add, concatenate,GlobalAveragePooling3D,Conv3D,LeakyReLU,ELU, MaxPooling3D,AveragePooling3D
from tensorflow.keras.regularizers import l2 as L2
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.optimizers import Adam, SGD,Adagrad
from keras.engine.topology import Layer
from keras.layers import BatchNormalization
from keras.layers import Input, InputLayer, InputSpec
#from tensorflow.keras import backend as K
#from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
#import tensorflow.compat.v1 as tf
# tf.compat.v1.enable_v2_behavior()
# tf.compat.v1.enable_eager_execution()

from tensorflow.compat.v1 import keras
from tensorflow.compat.v1.keras.layers import Dense, Dropout, Activation, Flatten, Input, add, concatenate,GlobalAveragePooling3D,Conv3D,LeakyReLU,ELU, MaxPooling3D,AveragePooling3D
from tensorflow.keras import layers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def getData(path):
  original_data = pd.read_excel(path)
  original_data.dropna(inplace=True)
  original_data.drop(["ID","Sex"], axis = 1, inplace=True)

  train, test = train_test_split(original_data, test_size = 0.2, random_state =100)


  x_train = train.drop(["Age"],axis=1).to_numpy(np.float32)
  y_train = train["Age"].to_numpy(np.float32)

  x_test = test.drop(["Age"],axis=1).to_numpy(np.float32)
  y_test = test["Age"].to_numpy(np.float32)

  return x_train, y_train, x_test, y_test

In [ ]:
x_train, y_trainm, x_test, y_test = getData('/content/drive/MyDrive/소융_캡스톤/뇌 MRI 사진_교수님/IXI_train_test_age.xlsx')

In [ ]:
num_train_image = len(x_train)
num_test_image= len(x_test)
print("Train set: ", num_train_image)
print("Test set: ", num_test_image)
print("Train set Shape: ", x_train.shape)

Train set:  370
Test set:  93
Train set Shape:  (370, 69)


In [ ]:
#client Data Non-iid 형태로 만들기 // 중복X

def Make_data_to_Non_RE_Client_data(num_train_image, Client_num, x_train, y_train):
  client_train_dataset = collections.OrderedDict()
  partition_Start = 0
  partition_End = int(num_train_image/Client_num - 1)

  for i in range(0, Client_num):
    client_key = "client_" + str(i+1) # client_1, client_2 .....

    train_data = collections.OrderedDict((('features', x_train[partition_Start:partition_End]),('label', y_train[partition_Start:partition_End])))
    client_train_dataset[client_key] = train_data
    partition_Start = partition_End + 1
    partition_End = int(partition_End + num_train_image/Client_num - 1)

  #tensorSliceClientData를 simulation의 instance로 변환
  train_dataset = tff.simulation.datasets.TestClientData(client_train_dataset)

  return train_dataset

In [ ]:
#client Data Non-iid 형태로 만들기 // 중복 허용

def Make_data_to_RE_Client_data(num_train_image, Client_num, x_train, y_train):
  client_train_dataset = collections.OrderedDict()
  Client_num = 4
  for i in range(0, Client_num):
    client_key = "client_" + str(i+1) # client_1, client_2 .....

  # non-iid를 위한 random sampling
    front_train = random.randint(1, num_train_image)
    rear_train = random.randint(front_train, num_train_image)

    train_data = collections.OrderedDict((('features', x_train[front_train:rear_train]),('label', y_train[front_train:rear_train])))
    client_train_dataset[client_key] = train_data

  #tensorSliceClientData를 simulation의 instance로 변환
  train_dataset = tff.simulation.datasets.TestClientData(client_train_dataset)

  return train_dataset

In [ ]:
#총 Client 수 결정

Original_Client_data = Make_data_to_Non_RE_Client_data(num_train_image, 4, x_train, y_train)

In [ ]:
# 각 Client 별로 학습할 데이터 추출

NUM_EPOCHS = 10
BATCH_SIZE = 100 #각 Client가 가질 데이터 수
SHUFFLE_BUFFER = 10
PREFETCH_BUFFER = 10

def preprocess(dataset):

  def batch_format_fn(element):
    return collections.OrderedDict(
        x=tf.reshape(element["features"], [-1,69]),
        y=tf.reshape(element["label"], [-1, 1]))

  return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(
      BATCH_SIZE).prefetch(PREFETCH_BUFFER).map(batch_format_fn)

In [ ]:
def make_federated_data(client_data, client_ids):
  result_list = []
  for id in client_ids:
    preprocess_data = preprocess(client_data.create_tf_dataset_for_client(id))
    sample = tf.nest.map_structure(lambda x: x.numpy(),next(iter(preprocess_data)))
    result_list.append(preprocess_data)
  print("Each client data size : ", len(sample["y"]))
  return result_list

In [ ]:
sample_clients = Original_Client_data.client_ids[0:4]

federated_train_data = make_federated_data(Original_Client_data, sample_clients)

print('Number of client datasets: {l}'.format(l=len(federated_train_data)))
print('First dataset: {d}'.format(d=federated_train_data[0]))
print("input_spec : {n}".format(n = federated_train_data[0].element_spec))

Each client data size :  100
Number of client datasets: 4
First dataset: <MapDataset shapes: OrderedDict([(x, (None, 69)), (y, (None, 1))]), types: OrderedDict([(x, tf.float32), (y, tf.float32)])>
input_spec : OrderedDict([('x', TensorSpec(shape=(None, 69), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))])


In [ ]:
def build_model():
  model = keras.Sequential([
   layers.Dense(1, input_shape=(69,), activation = 'linear'),
   #layers.Dense(10),
   #layers.Dense(5, activation= 'linear'),
   #layers.Dense(1, activation="linear")
  ])
  return model

In [ ]:
def model_fn():
    keras_model = build_model()
    return tff.learning.from_keras_model(
      keras_model,
      input_spec=federated_train_data[0].element_spec,
      loss=tf.keras.losses.MeanAbsoluteError(),
      metrics=[tf.keras.metrics.MeanAbsoluteError()])

In [ ]:
def initialize_fn():
  model = model_fn()
  return model.trainable_variables

In [ ]:
def next_fn(server_weights, federated_dataset):
  server_weights_at_client = broadcast(server_weights)
  client_weights = client_update(federated_dataset, server_weights_at_client)
  mean_client_weights = mean(client_weights)
  server_weights = server_update(mean_client_weights)
  
  return server_weights

In [ ]:
@tf.function
def client_update(model, dataset, server_weights, client_optimizer):
  print("client_update tf.fn")
  client_weights = model.trainable_variables
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        client_weights, server_weights)
  
  for batch in dataset:
      with tf.GradientTape() as tape:
        outputs = model.forward_pass(batch)

      grads = tape.gradient(outputs.loss, client_weights)
      grads_and_vars = zip(grads, client_weights)

      client_optimizer.apply_gradients(grads_and_vars)

  return client_weights

In [ ]:
@tf.function
def server_update(model, mean_client_weights):
  model_weights = model.trainable_variables
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        model_weights, mean_client_weights)
  return model_weights

In [ ]:
@tff.tf_computation
def server_init():
  model = model_fn()
  return model.trainable_variables

In [ ]:
@tff.federated_computation
def initialize_fn():
  return tff.federated_value(server_init(),tff.SERVER)

In [ ]:
tf_dataset_type = tff.SequenceType(model_fn().input_spec)

In [ ]:
str(tf_dataset_type)

'<x=float32[?,69],y=float32[?,1]>*'

In [ ]:
model_weights_type = server_init.type_signature.result

In [ ]:
str(model_weights_type)

'<float32[69,1],float32[1]>'

In [ ]:
@tff.tf_computation(tf_dataset_type, model_weights_type)
def client_update_fn(tf_dataset, server_weights):
  model = model_fn()
  client_optimizer = tf.keras.optimizers.SGD(learning_rate=0.01) #Adam, SGD
  return client_update(model, tf_dataset, server_weights, client_optimizer)

client_update tf.fn


In [ ]:
@tff.tf_computation(model_weights_type)
def server_update_fn(mean_client_weights):
  model = model_fn()
  return server_update(model, mean_client_weights)

In [ ]:
federated_server_type = tff.FederatedType(model_weights_type, tff.SERVER)
federated_dataset_type = tff.FederatedType(tf_dataset_type, tff.CLIENTS)

In [ ]:
str(federated_dataset_type)

'{<x=float32[?,69],y=float32[?,1]>*}@CLIENTS'

In [ ]:
str(federated_server_type)

'<float32[69,1],float32[1]>@SERVER'

In [ ]:
# 양자화 가능!!!
@tff.federated_computation(federated_server_type, federated_dataset_type)
def next_fn(server_weights, federated_dataset):
  server_weights_at_client = tff.federated_broadcast(server_weights)
  client_weights = tff.federated_map(
        client_update_fn, (federated_dataset, server_weights_at_client))
  mean_client_weights = tff.federated_mean(client_weights)
  server_weights = tff.federated_map(server_update_fn, mean_client_weights)

  return server_weights

In [ ]:
#iterative_process = tff.learning.build_federated_averaging_process(
#    model_fn,
#    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.02),
#    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

In [ ]:
#str(iterative_process.initialize.type_signature)

In [ ]:
federated_algorithm = tff.templates.IterativeProcess(
      initialize_fn=initialize_fn,
      next_fn=next_fn
    )

In [ ]:
def evaluate(server_state):
  keras_model = build_model()
  keras_model.compile(
      loss=tf.keras.losses.MeanAbsoluteError(),
      metrics=[tf.keras.metrics.MeanAbsoluteError()]
  )
  results = keras_model.evaluate(x_test, y_test)

In [ ]:
server_state = federated_algorithm.initialize()

In [ ]:
#state = iterative_process.initialize()

In [ ]:
evaluate(server_state)
#evaluate(state)

3/3 [==============================] - 0s 2ms/step - loss: 472.2916 - mean_absolute_error: 472.2916


In [ ]:
for round in range(100):
  print("round ",round+1)
  server_state = federated_algorithm.next(server_state, federated_train_data)
  #state = iterative_process.next(state, federated_train_data)
  evaluate(server_state)
  #evaluate(state)
  print("")

round  1
3/3 [==============================] - 0s 3ms/step - loss: 180.8625 - mean_absolute_error: 180.8625

round  2
3/3 [==============================] - 0s 3ms/step - loss: 456.8864 - mean_absolute_error: 456.8864

round  3
3/3 [==============================] - 0s 2ms/step - loss: 204.9520 - mean_absolute_error: 204.9520

round  4
3/3 [==============================] - 0s 3ms/step - loss: 426.3410 - mean_absolute_error: 426.3410

round  5
3/3 [==============================] - 0s 3ms/step - loss: 285.2712 - mean_absolute_error: 285.2712

round  6
3/3 [==============================] - 0s 3ms/step - loss: 224.9111 - mean_absolute_error: 224.9111

round  7
3/3 [==============================] - 0s 2ms/step - loss: 26.9128 - mean_absolute_error: 26.9128

round  8
3/3 [==============================] - 0s 3ms/step - loss: 320.3269 - mean_absolute_error: 320.3269

round  9
3/3 [==============================] - 0s 2ms/step - loss: 27.1643 - mean_absolute_error: 27.1643

round  10
3/3 

In [ ]:
evaluate(server_state)

3/3 [==============================] - 0s 3ms/step - loss: 354.7229 - mean_absolute_error: 354.7229
